# Mongochain: Single Agent Demo

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/robin-mongodb/mongochain/blob/main/examples/getting_started/single_agent_demo.ipynb)

This notebook demonstrates the core capabilities of a single `MongoAgent`:
- Creating an agent with MongoDB-backed memory
- Chatting with user-specific context
- Automatic fact extraction and storage
- Memory retrieval and management

## Prerequisites
- MongoDB Atlas cluster (with vector search support)
- Voyage AI API key (for embeddings)
- LLM API key (OpenAI, Anthropic, or Google)

## 1. Install Dependencies

Run this cell to install the required packages:

In [ ]:
# Install mongochain from GitHub (includes all dependencies)
%pip install git+https://github.com/robin-mongodb/mongochain.git

## 2. Configure API Keys

Enter your API keys when prompted (input will be hidden):

In [ ]:
from getpass import getpass

# MongoDB Atlas connection string
MONGO_URI = getpass("MongoDB URI: ")

# Voyage AI API key (for embeddings)
VOYAGE_API_KEY = getpass("Voyage API Key: ")

# LLM API key (OpenAI in this example)
OPENAI_API_KEY = getpass("OpenAI API Key: ")

## 3. Create the Agent

Create a `MongoAgent` with a name and persona. The agent will automatically:
- Create a MongoDB database named after the agent
- Set up collections for conversation history, short-term, and long-term memory
- Create vector search indexes for semantic retrieval

In [ ]:
from mongochain import MongoAgent

# Create the agent
agent = MongoAgent(
    name="research_assistant",
    persona="""You are a helpful research assistant with expertise in technology and software development.
You remember details about users and personalize your responses based on their background and interests.
You are friendly, concise, and always eager to help.""",
    mongo_uri=MONGO_URI,
    voyage_api_key=VOYAGE_API_KEY,
    llm_api_key=OPENAI_API_KEY,
    llm_provider="openai",  # Options: "openai", "anthropic", "google"
)

print(f"\n{agent}")

## 4. Basic Chat

Send a message to the agent. Include some personal details — the agent will automatically extract and remember facts about you.

In [ ]:
# Define a user ID (typically an email)
USER_ID = "demo@example.com"

# Send a message with some personal context
response = agent.chat(
    user_id=USER_ID,
    message="Hi! I'm a DevOps engineer at Acme Corp. I'm currently working on optimizing our MongoDB deployment for better performance."
)

print(response)

## 5. Change Agent Persona

Dynamically change the agent's personality. The new persona is persisted to MongoDB.

In [ ]:
# Change to a cynical persona
agent.set_persona("""You are a world-weary tech veteran who has seen it all.
You're cynical, curt, and delightfully sarcastic — but deep down you actually want to help.
Keep responses brief. Roll your eyes at obvious questions but still answer them.
You remember everything about the user and use it to make pointed observations.""")

# Chat with the new persona
response = agent.chat(
    user_id=USER_ID,
    message="Can you help me understand MongoDB indexing strategies?"
)

print(response)

## 6. Custom Tools

Register a custom function as a tool. The agent will automatically use it when appropriate.

Tools are persisted to MongoDB in two places:
- `agent_tools` collection — full tool definitions
- `agent_metadata.tools` — list of tool names

In [ ]:
import requests

def get_weather(location: str, unit: str = "celsius") -> str:
    """Get current weather for a location using Open-Meteo API (free, no API key)."""
    try:
        # Step 1: Geocode the location to get coordinates
        geo_url = f"https://geocoding-api.open-meteo.com/v1/search?name={location}&count=1"
        geo_response = requests.get(geo_url, timeout=10)
        geo_data = geo_response.json()
        
        if "results" not in geo_data or not geo_data["results"]:
            return f"Could not find location: {location}"
        
        lat = geo_data["results"][0]["latitude"]
        lon = geo_data["results"][0]["longitude"]
        city_name = geo_data["results"][0]["name"]
        country = geo_data["results"][0].get("country", "")
        
        # Step 2: Get current weather
        temp_unit = "fahrenheit" if unit == "fahrenheit" else "celsius"
        weather_url = (
            f"https://api.open-meteo.com/v1/forecast?"
            f"latitude={lat}&longitude={lon}"
            f"&current=temperature_2m,weather_code,wind_speed_10m"
            f"&temperature_unit={temp_unit}"
        )
        weather_response = requests.get(weather_url, timeout=10)
        weather_data = weather_response.json()
        
        current = weather_data["current"]
        temp = current["temperature_2m"]
        wind = current["wind_speed_10m"]
        
        # Weather code to description
        weather_codes = {
            0: "Clear sky", 1: "Mainly clear", 2: "Partly cloudy", 3: "Overcast",
            45: "Foggy", 48: "Depositing rime fog",
            51: "Light drizzle", 53: "Moderate drizzle", 55: "Dense drizzle",
            61: "Slight rain", 63: "Moderate rain", 65: "Heavy rain",
            71: "Slight snow", 73: "Moderate snow", 75: "Heavy snow",
            80: "Slight rain showers", 81: "Moderate rain showers", 82: "Violent rain showers",
            95: "Thunderstorm", 96: "Thunderstorm with slight hail", 99: "Thunderstorm with heavy hail"
        }
        condition = weather_codes.get(current["weather_code"], "Unknown")
        symbol = "°F" if unit == "fahrenheit" else "°C"
        
        return f"Weather in {city_name}, {country}: {condition}, {temp}{symbol}, Wind: {wind} km/h"
    
    except Exception as e:
        return f"Error fetching weather: {str(e)}"

# Register the tool with the agent
agent.register_tool(
    name="get_weather",
    func=get_weather,
    description="Get the current weather for a city. Use this when the user asks about weather conditions.",
    parameters={
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "City name, e.g., 'London', 'New York'"
            },
            "unit": {
                "type": "string",
                "enum": ["celsius", "fahrenheit"],
                "description": "Temperature unit (default: celsius)"
            }
        },
        "required": ["location"]
    }
)

In [ ]:
# Ask a question that will trigger the tool
response = agent.chat(
    user_id=USER_ID,
    message="What's the weather like in London right now?"
)

print(response)

## 7. Memory-Aware Follow-ups

The agent automatically remembers facts about the user from previous conversations. It uses this context to personalize responses — even without explicit reminders.

In [ ]:
# Ask a follow-up question that relies on the agent remembering user context
# Note: We never mentioned our job or company in this message!
response = agent.chat(
    user_id=USER_ID,
    message="Based on what you know about me, what MongoDB features should I prioritize learning?"
)

print(response)

In [ ]:
# Another follow-up — the agent should reference our role and company
response = agent.chat(
    user_id=USER_ID,
    message="Can you summarize what you know about me so far?"
)

print(response)